In [3]:
from utils.bech32m import convertbits, bech32_encode, Encoding
from utils.key import ECKey, ECPubKey, generate_bip340_key_pair
from utils.bip158 import gcs_match_any

from bip32 import BIP32
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException

import hashlib
import secp256k1

def sha256(s):
    return hashlib.new('sha256', s).digest()

# Silent Payments

Using a new address for each Bitcoin transaction is a crucial aspect of maintaining privacy. This often requires a secure interaction between sender and receiver so that the receiver can hand out a fresh address, a batch of fresh address, or a way for the send to generate addresses, such as an xpub.

However, interaction is often infeasible and in many cases undesirable. To solve for this, various protocols have been proposed which use a static payment address and notifications, sent via the blockchain[footnote]. These protocols eliminate the need for interaction, but at the expense of increased costs for one-time payments and a noticeable footprint in the blockchain, potentially revealing metadata about the sender and receiver. Notification schemes also allow the receiver to link all payments from the same sender, compromising sender privacy.

This proposal aims to address the limitations of these current approaches by presenting a solution that eliminates the need for interaction, eliminates the need for notifications, and protects both sender and receiver privacy.

## Goals

We aim to present a transaction protocol which satisifies the following properties:

* No increase in the size or cost of transactions
* Resulting transactions blend in with other bitcoin transactions and can't be distinguished
* Transactions can't be linked to a silent payment address by an outside observer
* No sender-receiver interaction required
* No linking of multiple payments to the same sender
* Each silent payment goes to a unique address, avoiding accidental reuse
* Supports payment purpose labeling
* Uses existing seed phrase or descriptor methods for backup and recovery
* Separates scanning and spending responsibilities
* Compatible with other spending protocols, such as CoinJoin
* Light client/SPV wallet support
* Protocol is upgradeable


## What this workshop will attempt to cover

* Generating a silent payment address
* Scanning for silent payments
* Sync the wallet using BIP158 block filters
* BONUS:
  * Add label support
  * Scan for multiple outputs
  * Run on signet
  
## What this workshop won't cover

* Sending to silent payment addresses

# Elliptic Curve math review

Elliptic Curve math involves scalars and points.

* A scalar is a positive integer which is smaller than the group order, and is denoted by a lower case letter (eg `a`).
* A point lies on the curve and is denoted by an upper-case letter (eg `C`) or a pair of co-ordinates (eg `(x,y)`).

In Bitcoin, key pair generation and signing is performed over the secp256k1 curve. All scalars are modulo the group order `SECP256K1_ORDER`, which is a very large number

![test](images/ec_math0.jpg)

_An overview of all operations of scalars and points over elliptic curves._

## Simple case

Alice publishes a public key *A* as her silent payment address. Bob selects an input *B,b* and tweaks Alice's public key to create a destination public key *D* in the following way: 

* Let *D = HASH(b·A)·G + A* 

Bob constructs a transaction in the normal way with *B* as an input and *D* as the destination address. Alice detects this payment by computing:

* Let *D = HASH(a·B)·G + A* (Diffie-Hellman Key Exchange).


In [10]:
b, B = generate_bip340_key_pair()
a, A = generate_bip340_key_pair()

# Bob generates the output using his private key b and Alice's public key
# sha256(a * B) * G + A
t = sha256((b * A).get_bytes())
T = ECKey().set(t).get_pubkey()
D = T + A

# Alice checks if D is hers using her private key and Bob's public key
t_prime = sha256((a * B).get_bytes())
T_prime = ECKey().set(t_prime).get_pubkey()
D_prime = T_prime + A

assert D == D_prime

## Preventing address reuse

If Bob were to use a different UTXO from the same public key *B* for a subseqent payment, he would end up deriving the same destination for Alice. To prevent this, Bob must include a hash of the outpoint in the following manner:

* Let *outpoint_hash = HASH(txid \|\| vout)* 
* Let *D~0~ = HASH(outpoint_hash·b·A)·G + A*


In [ ]:
# calculate the outpoint hash

txid = ""
vout = ""

# D = sha256(outpoint_hash * b * A) * G + A

In [2]:

class SPWallet:
    
    REASON = "1337"

    def __init__(self, seed):
        self.master = BIP32.from_seed(bytes.fromhex(seed))
        self.spend_path = f"m/{self.REASON}'/0'/0'"
        self.scan_path = f"m/{self.REASON}'/1'/0'"
        self.scan_privkey, self.scan_pubkey = self.convert_to_bip340_key_pair(
            self.get_scan_privkey()
        )
        self.spend_privkey, self.spend_pubkey = self.convert_to_bip340_key_pair(
            self.get_spend_privkey()
        )
        
    def convert_to_bip340_key_pair(self, seckey):
        d = ECKey().set(seckey)
        P = d.get_pubkey()
        if P.get_y()%2 != 0:
            d.negate()
            P.negate()
        return d, P
        
    def get_scan_pubkey(self):
        return self.master.get_pubkey_from_path(self.scan_path)
    
    def get_scan_privkey(self):
        return self.master.get_privkey_from_path(self.scan_path)
    
    def get_spend_pubkey(self):
        return self.master.get_pubkey_from_path(self.spend_path)
    
    def get_spend_privkey(self):
        return self.master.get_privkey_from_path(self.spend_path)
    
    def get_silent_payment_address(self):
        pubkeys = self.scan_pubkey.get_bytes() + self.spend_pubkey.get_bytes()
        data = convertbits(pubkeys, 8, 5)
        return bech32_encode("sprt", data, Encoding.BECH32M)

In [3]:
class SPScanner:
    PUBKEY_BYTES = 33
    TRUNC_HASH_BYTES = 8
    WITNESS_VERSION_1 = '5120'
    
    def __init__(self, spend_pubkey, scan_privkey, rpc_client, start_height):

        self.spend_pubkey = spend_pubkey
        self.scan_privkey = scan_privkey
        self.start_from_height = start_height
        self.last_block_scanned = self.start_from_height - 1
        self.rpc = rpc_client
        
    def refresh(self, rpc_client):
        self.rpc = rpc_client
        
    def scan(self, start=None):
    
        if start:
            self.start_from_height = start
            
        # get the chain height
        stop = self.rpc.getblockchaininfo()['blocks']
        current_block = self.start_from_height
        while current_block <= stop:
            res = self.get_silent_payment_block_data(current_block)
            if res['total_tx'] == 0:
                current_block += 1
                continue
            
            tweak_data = self.parse_silent_payment_block_data(res)
            outputs_to_check = self.compute_outputs(tweak_data)
            self.is_mine(outputs_to_check, res['block_hash'])
            current_block += 1
            
        self.start_from_height = current_block
        print("done scanning")
        
    def get_silent_payment_block_data(self, height):
            
        block_hash = self.rpc.getblockhash(height)
        data = self.rpc.getsilentpaymentblockdata(block_hash)
        data['block_hash'] = block_hash
        return data
        
    def parse_silent_payment_block_data(self, data):
        total_txs = data['total_tx']
        tx_data = data['data']
        txs = [
            tx_data[i:i + (self.PUBKEY_BYTES + self.TRUNC_HASH_BYTES)*2]
            for i in range(0, len(tx_data), (self.PUBKEY_BYTES + self.TRUNC_HASH_BYTES)*2)
        ]
        tweaks = [
            (tx[:self.PUBKEY_BYTES*2], tx[self.PUBKEY_BYTES*2:]) for tx in txs
        ]
        assert len(tweaks) == total_txs
        return tweaks
    
    def compute_outputs(self, txs):
        potential_outputs = []
        for (sum_pubkeys, trunc_hash) in txs:
            # compute the outpoint hash tweak
            outpoint_hash = sha256(bytes.fromhex(trunc_hash))
            
            # scalar multiplication with scan_privkey
            scalar_tweak = outpoint_hash * self.scan_privkey
            
            # tweaked pubkey - expensive EC Multiplication
            # this needs to be x-only
            secp256k1_pubkey = secp256k1.PublicKey(bytes.fromhex(sum_pubkeys), raw=True)
            shared_secret = secp256k1_pubkey.ecdh(scalar_tweak.get_bytes())
            # shared_secret_hash = sha256(shared_secret)
            pubkey = ECKey().set(shared_secret).get_pubkey() + self.spend_pubkey
            
            # create a taproot scriptPubKey
            spk = self.WITNESS_VERSION_1 + pubkey.get_bytes().hex()
            potential_outputs.append(spk)
            
        return potential_outputs
    
    def get_compact_block_filter(self, block_hash):
        res = self.rpc.getblockfilter(block_hash)
        
        # cheating here, because this is actually a compactSize field
        N = int(res['filter'][:2], 16)
        block_filter = res['filter'][2:]
        return {'size': N, 'filter': block_filter}
    
    def is_mine(self, script_pub_keys, block_hash):
        
        gcs_data = self.get_compact_block_filter(block_hash)
        if gcs_match_any(block_hash, script_pub_keys, gcs_data):
            block_data = self.rpc.getblock(block_hash, 2)
            for tx in block_data['tx']:
                for vout in tx['vout']:
                    if vout['scriptPubKey']['hex'] in script_pub_keys:
                        print(f"{tx['txid']} is mine!")
        
def rpc_client():
    return AuthServiceProxy("http://%s:%s@127.0.0.1:18443"%("test", "test"))

wallet = SPWallet(sha256(b'stuff n fluff').hex())
scanner = SPScanner(wallet.spend_pubkey, wallet.scan_privkey, rpc_client(), 315)

In [4]:
wallet.get_silent_payment_address()
scanner.scan()

f59fe48b899ee2448ff45761f268ef5ae2d461974d987cc12534b69f03c9f468 is mine!
done scanning
